# **Notebook to create joint table**

In this notebook we will create a query between the tables. the result of the join will be added as temp delta table.  To use it properly , please follow instructions. 

# Instructions 
- please replace the lakehouse name with your own

In [ ]:
# Lakehouse name , replace with your own
lakeHouseName = "dataverse_development_cds2_workspace_unqf0798579be6eee118bc36045bd003"

# SQL Query
query = f"""
SELECT 
    c.crffa_cif,c.crffa_revenue,c.crffa_csat,c.crffa_noofreturns,c.crffa_educationalbackground,c.crffa_yearofbirth,c.crffa_recency,
    r.MntWines,r.MntFruits, r.MntMeatProducts,r.MntFishProducts,r.MntBakeryProducts,r.MntBeverageProds,r.MntDairyProds,r.NumDealsPurchases,r.NumWebPurchases,
    r.NumCatalogPurchases,r.NumStorePurchases
FROM 
    {lakeHouseName}.contact AS c
JOIN 
    {lakeHouseName}.retailstoretxnsummary_01 AS r
ON 
    c.crffa_cif = r.ID
WHERE 
    c.crffa_dateforarticle = 1
"""

# Execute the query
dfJoint = spark.sql(query)

# Show the DataFrame
dfJoint.printSchema()

In [ ]:
# drop CIF as it is not a valuable input to the model. 
dfJoint = dfJoint.drop('crffa_cif')

# Save the DataFrame to the Data Lake
table_name = "tbl_temp_traintestdata"
dfJoint.write.format('delta').mode("overwrite").save(f"Tables/{table_name}")
print(f"Spark DataFrame saved to delta table: {table_name} for train and test")